# Imports

In [2]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [15]:
import pandas as pd
from google.cloud import bigquery
import plaid
from utils.bq_utils import BqUtils
from utils.plaid_utils import PlaidUtils
from sql.bq_table_schemas import BqTableSchemas

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)


PLAID_CLIENT_ID = "65975384ab670e001c0aaf0d"
# PLAID_SECRET="56e33c77237c8c9e45f5c066b8b2fa" #production
PLAID_SECRET = "9294dd5ca4a5c99d90da56640f40e5"  # sandbox
PLAID_ENV = "sandbox"
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
PLAID_HOST = plaid.Environment.Sandbox
# PLAID_REDIRECT_URI="https://localhost:3000/"
PLAID_ACCESS_TOKENS = [
    "access-sandbox-1902b6d6-33f3-49b8-a3ce-3213a5fe9a49",
    "access-sandbox-12481aea-33b6-4101-9f49-4e07ebbc1431",
    "access-sandbox-f415798a-74af-4e7f-b504-dc075dc5889c",
]


bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
bq_tables = BqTableSchemas()

plaid_cursors_bq = bq_tables.plaid_cursors_YYYYMMDD()

# Test Transactions Sync

In [4]:
plaid_client.create_temp_cursors_bq_table(confirm=False)

latest_cursors_df = plaid_client.get_latest_cursors()

print(latest_cursors_df["access_token"][0])
transactions_df, removed_df = plaid_client.get_transactions(
    access_token=latest_cursors_df["access_token"][0],
    item_id=latest_cursors_df["item_id"][0],
    next_cursor=latest_cursors_df["next_cursor"][0],
)

SUCCESS: `zsc-personal.personal_finance.temp_plaid_cursors` successfully deleted!
SUCCESS: `zsc-personal.personal_finance.temp_plaid_cursors` successfully created!
access-sandbox-f415798a-74af-4e7f-b504-dc075dc5889c
Cursors successfully added to `zsc-personal.personal_finance.temp_plaid_cursors`
SUCCESS: retrieved transacstions!


In [16]:
transactions_df.head()

,account_id,account_owner,status,amount,authorized_date,authorized_datetime,category,category_id,check_number,counterparties,date,datetime,currency_code,address,city,region,postal_code,latitude,longitude,merchant_entity_id,merchant_name,name,payment_channel,reference_number,ppd_id,payee,by_order_of,payer,payment_method,payment_processor,reason,pending,pending_transaction_id,personal_finance_category_confidence_level,personal_finance_category_detailed,personal_finance_category_primary,transaction_code,transaction_id,transaction_type,unofficial_currency_code,website
0,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,6.33,2024-04-06,None,"[Travel, Taxi]",22016000,None,"[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]",2024-04-07,None,USD,None,None,None,None,None,None,eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd,Uber,Uber 072515 SF**POOL**,online,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,TRANSPORTATION,None,EG4wZPMApWSkLDaE4k88tlwmyZADWrC4e7W7x,special,None,uber.com
1,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,5.40,2024-03-24,None,"[Travel, Taxi]",22016000,None,"[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]",2024-03-25,None,USD,None,None,None,None,None,None,eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd,Uber,Uber 063015 SF**POOL**,online,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,TRANSPORTATION,None,WvGLrb6Z7AtkboXnMkWWtPdVBWjvG3F6y8e8z,special,None,uber.com
2,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,-500.00,2024-03-23,None,"[Travel, Airlines and Aviation Services]",22001000,None,"[{'name': 'United Airlines', 'type': 'merchant', 'website': 'united.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/united_airlines_1065.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4', 'phone_number': None}]",2024-03-23,None,USD,None,None,None,None,None,None,NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4,United Airlines,United Airlines,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRAVEL_FLIGHTS,TRAVEL,None,A83xjd7gZWtKmNlvPKeetXEl8AP3W7t9JDqDQ,special,None,united.com
3,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,12.00,2024-03-22,None,"[Food and Drink, Restaurants, Fast Food]",13005032,None,"[{'name': 'McDonald's', 'type': 'merchant', 'website': 'mcdonalds.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/mcdonalds_619.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22', 'phone_number': None}]",2024-03-22,None,USD,None,None,None,None,None,None,vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22,McDonald's,McDonald's,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,FOOD_AND_DRINK_FAST_FOOD,FOOD_AND_DRINK,None,GvPy17Lqg6tK8LdGWKEEtGqM8K6JAPi6xvDv4,place,None,mcdonalds.com
4,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,4.33,2024-03-22,None,"[Food and Drink, Restaurants, Coffee Shop]",13005043,None,"[{'name': 'Starbucks', 'type': 'merchant', 'website': 'starbucks.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/starbucks_956.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NZAJQ5wYdo1W1p39X5q5gpb15OMe39pj4pJBb', 'phone_number': None}]",2024-03-22,None,USD,None,None,None,None,None,None,NZAJQ5wYdo1W1p39X5q5gpb15OMe39pj4pJBb,Starbucks,Starbucks,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,FOOD_AND_DRINK_COFFEE,FOOD_AND_DRINK,None,naVMWNLPG9CGQlvjJGqqtXDeGdv36EtAVR1Ry,place,None,starbucks.com


In [7]:
for i in transactions_df["counterparties"]:
    print(i)

[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]
[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]
[{'name': 'United Airlines', 'type': 'merchant', 'website': 'united.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/united_airlines_1065.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4', 'phone_number': None}]
[{'name': "McDonald's", 'type': 'merchant', 'website': 'mcdonalds.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/mcdonalds_619.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22', 'phone_number': None}]
[{'na

In [18]:
transactions_df[~transactions_df["merchant_name"].isnull()].head()

,account_id,account_owner,status,amount,authorized_date,authorized_datetime,category,category_id,check_number,counterparties,date,datetime,currency_code,address,city,region,postal_code,latitude,longitude,merchant_entity_id,merchant_name,name,payment_channel,reference_number,ppd_id,payee,by_order_of,payer,payment_method,payment_processor,reason,pending,pending_transaction_id,personal_finance_category_confidence_level,personal_finance_category_detailed,personal_finance_category_primary,transaction_code,transaction_id,transaction_type,unofficial_currency_code,website
0,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,6.33,2024-04-06,None,"[Travel, Taxi]",22016000,None,"[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]",2024-04-07,None,USD,None,None,None,None,None,None,eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd,Uber,Uber 072515 SF**POOL**,online,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,TRANSPORTATION,None,EG4wZPMApWSkLDaE4k88tlwmyZADWrC4e7W7x,special,None,uber.com
1,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,5.40,2024-03-24,None,"[Travel, Taxi]",22016000,None,"[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]",2024-03-25,None,USD,None,None,None,None,None,None,eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd,Uber,Uber 063015 SF**POOL**,online,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,TRANSPORTATION,None,WvGLrb6Z7AtkboXnMkWWtPdVBWjvG3F6y8e8z,special,None,uber.com
2,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,-500.00,2024-03-23,None,"[Travel, Airlines and Aviation Services]",22001000,None,"[{'name': 'United Airlines', 'type': 'merchant', 'website': 'united.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/united_airlines_1065.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4', 'phone_number': None}]",2024-03-23,None,USD,None,None,None,None,None,None,NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4,United Airlines,United Airlines,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRAVEL_FLIGHTS,TRAVEL,None,A83xjd7gZWtKmNlvPKeetXEl8AP3W7t9JDqDQ,special,None,united.com
3,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,12.00,2024-03-22,None,"[Food and Drink, Restaurants, Fast Food]",13005032,None,"[{'name': 'McDonald's', 'type': 'merchant', 'website': 'mcdonalds.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/mcdonalds_619.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22', 'phone_number': None}]",2024-03-22,None,USD,None,None,None,None,None,None,vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22,McDonald's,McDonald's,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,FOOD_AND_DRINK_FAST_FOOD,FOOD_AND_DRINK,None,GvPy17Lqg6tK8LdGWKEEtGqM8K6JAPi6xvDv4,place,None,mcdonalds.com
4,o4VJAPpjGouG4q8yZGKKtPrBko3PB9ioQDB7Q,None,ADDED,4.33,2024-03-22,None,"[Food and Drink, Restaurants, Coffee Shop]",13005043,None,"[{'name': 'Starbucks', 'type': 'merchant', 'website': 'starbucks.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/starbucks_956.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NZAJQ5wYdo1W1p39X5q5gpb15OMe39pj4pJBb', 'phone_number': None}]",2024-03-22,None,USD,None,None,None,None,None,None,NZAJQ5wYdo1W1p39X5q5gpb15OMe39pj4pJBb,Starbucks,Starbucks,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,FOOD_AND_DRINK_COFFEE,FOOD_AND_DRINK,None,naVMWNLPG9CGQlvjJGqqtXDeGdv36EtAVR1Ry,place,None,starbucks.com


In [21]:
transactions_df.groupby(["personal_finance_category_primary", "personal_finance_category_detailed"], as_index=False)[
    "transaction_id"
].nunique()

,personal_finance_category_primary,personal_finance_category_detailed,transaction_id
0,ENTERTAINMENT,ENTERTAINMENT_SPORTING_EVENTS_AMUSEMENT_PARKS_AND_MUSEUMS,1
1,FOOD_AND_DRINK,FOOD_AND_DRINK_COFFEE,3
2,FOOD_AND_DRINK,FOOD_AND_DRINK_FAST_FOOD,3
3,GENERAL_MERCHANDISE,GENERAL_MERCHANDISE_OTHER_GENERAL_MERCHANDISE,2
4,INCOME,INCOME_WAGES,3
5,LOAN_PAYMENTS,LOAN_PAYMENTS_CREDIT_CARD_PAYMENT,3
6,TRANSPORTATION,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,6
7,TRAVEL,TRAVEL_FLIGHTS,3


In [22]:
transactions_df["personal_detailed"] = transactions_df.apply(
    lambda row: row["personal_finance_category_detailed"][len(row["personal_finance_category_primary"]) + 1 :], axis=1
)

transactions_df.groupby(
    ["personal_finance_category_primary", "personal_finance_category_detailed", "personal_detailed"], as_index=False
)["transaction_id"].nunique()

,personal_finance_category_primary,personal_finance_category_detailed,personal_detailed,transaction_id
0,ENTERTAINMENT,ENTERTAINMENT_SPORTING_EVENTS_AMUSEMENT_PARKS_AND_MUSEUMS,SPORTING_EVENTS_AMUSEMENT_PARKS_AND_MUSEUMS,1
1,FOOD_AND_DRINK,FOOD_AND_DRINK_COFFEE,COFFEE,3
2,FOOD_AND_DRINK,FOOD_AND_DRINK_FAST_FOOD,FAST_FOOD,3
3,GENERAL_MERCHANDISE,GENERAL_MERCHANDISE_OTHER_GENERAL_MERCHANDISE,OTHER_GENERAL_MERCHANDISE,2
4,INCOME,INCOME_WAGES,WAGES,3
5,LOAN_PAYMENTS,LOAN_PAYMENTS_CREDIT_CARD_PAYMENT,CREDIT_CARD_PAYMENT,3
6,TRANSPORTATION,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,TAXIS_AND_RIDE_SHARES,6
7,TRAVEL,TRAVEL_FLIGHTS,FLIGHTS,3
